In [1]:
# !pip install -q numpy==1.19.5

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [28]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit
from sklearn import metrics

In [29]:
# setting to see all columns 
pd.set_option('display.max_columns', None)

# Load data and set input/output paths

In [30]:
from os.path import exists
# config: depends whether you're on Google Colab or local


# Get URL from github csv by clicking on Download > Copy Link Address

load_from_google_drive = False

if load_from_google_drive:
      # On google colab
      # Mount GDrive and attach it to the colab for data I/O
    from google.colab import drive
    drive.mount('/content/drive')
    input_dir = '/content/drive/My Drive/datum/vfp/data/input/'
    output_dir = '/content/drive/My Drive/datum/vfp/data/output/'
    os.makedirs(output_dir, exist_ok=True)

else:
  # If using jupyter-lab or jupyter notebook, load locally:
  input_dir = './data/input/'
  output_dir = './data/output/'
  output_dir = './data/output/bias_mitigation/'
  os.makedirs(output_dir, exist_ok = True)

import datetime
ts = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')


In [31]:
df = pd.read_csv(input_dir + 'features/egemaps_vector_speech_duration.csv', index_col = 0)
df

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,loudness_sma3_amean,loudness_sma3_stddevNorm,loudness_sma3_percentile20.0,loudness_sma3_percentile50.0,loudness_sma3_percentile80.0,loudness_sma3_pctlrange0-2,loudness_sma3_meanRisingSlope,loudness_sma3_stddevRisingSlope,loudness_sma3_meanFallingSlope,loudness_sma3_stddevFallingSlope,spectralFlux_sma3_amean,spectralFlux_sma3_stddevNorm,mfcc1_sma3_amean,mfcc1_sma3_stddevNorm,mfcc2_sma3_amean,mfcc2_sma3_stddevNorm,mfcc3_sma3_amean,mfcc3_sma3_stddevNorm,mfcc4_sma3_amean,mfcc4_sma3_stddevNorm,jitterLocal_sma3nz_amean,jitterLocal_sma3nz_stddevNorm,shimmerLocaldB_sma3nz_amean,shimmerLocaldB_sma3nz_stddevNorm,HNRdBACF_sma3nz_amean,HNRdBACF_sma3nz_stddevNorm,logRelF0-H1-H2_sma3nz_amean,logRelF0-H1-H2_sma3nz_stddevNorm,logRelF0-H1-A3_sma3nz_amean,logRelF0-H1-A3_sma3nz_stddevNorm,F1frequency_sma3nz_amean,F1frequency_sma3nz_stddevNorm,F1bandwidth_sma3nz_amean,F1bandwidth_sma3nz_stddevNorm,F1amplitudeLogRelF0_sma3nz_amean,F1amplitudeLogRelF0_sma3nz_stddevNorm,F2frequency_sma3nz_amean,F2frequency_sma3nz_stddevNorm,F2bandwidth_sma3nz_amean,F2bandwidth_sma3nz_stddevNorm,F2amplitudeLogRelF0_sma3nz_amean,F2amplitudeLogRelF0_sma3nz_stddevNorm,F3frequency_sma3nz_amean,F3frequency_sma3nz_stddevNorm,F3bandwidth_sma3nz_amean,F3bandwidth_sma3nz_stddevNorm,F3amplitudeLogRelF0_sma3nz_amean,F3amplitudeLogRelF0_sma3nz_stddevNorm,alphaRatioV_sma3nz_amean,alphaRatioV_sma3nz_stddevNorm,hammarbergIndexV_sma3nz_amean,hammarbergIndexV_sma3nz_stddevNorm,slopeV0-500_sma3nz_amean,slopeV0-500_sma3nz_stddevNorm,slopeV500-1500_sma3nz_amean,slopeV500-1500_sma3nz_stddevNorm,spectralFluxV_sma3nz_amean,spectralFluxV_sma3nz_stddevNorm,mfcc1V_sma3nz_amean,mfcc1V_sma3nz_stddevNorm,mfcc2V_sma3nz_amean,mfcc2V_sma3nz_stddevNorm,mfcc3V_sma3nz_amean,mfcc3V_sma3nz_stddevNorm,mfcc4V_sma3nz_amean,mfcc4V_sma3nz_stddevNorm,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,sid,token,target,filename,duration,task
0,38.72469,0.074773,37.18876,38.47970,40.44019,3.251423,37.94022,15.84932,206.45270,360.75950,0.409657,0.516380,0.211142,0.385909,0.588256,0.377114,5.487913,2.772103,5.275577,2.848066,0.183393,0.628117,18.73326,0.802712,10.520690,1.304518,7.572035,1.962388,-4.375561,-3.278681,0.023441,1.147700,0.908820,0.858329,10.061590,0.348174,19.314520,0.650321,21.89401,0.478417,530.5170,0.394628,1226.236,0.232456,-79.00346,-1.113559,1590.281,0.167123,784.5121,0.407070,-65.53321,-1.063740,2636.545,0.095774,703.0072,0.460240,-67.30154,-1.014580,-16.121190,-0.486312,24.37623,0.353210,0.041570,0.867909,-0.014989,-0.643631,0.202319,0.572868,24.18615,0.417486,8.214763,1.525709,7.750072,2.055495,-7.210148,-2.001652,-6.364767,13.05844,-0.020086,-0.002822,0.131253,6.017192,2.325582,0.323750,0.279148,0.086250,0.056111,-36.04536,VFP10,Speech1,1,VFP10_Speech1,3.503,Speech
1,41.11026,0.121323,37.82011,40.29837,42.86220,5.042088,65.27183,67.44999,107.02950,136.12460,0.352636,0.772619,0.091338,0.276297,0.643402,0.552064,5.773097,3.043491,5.736364,3.096833,0.165227,1.078465,16.27722,0.886090,4.175593,2.418372,4.164841,4.175746,-1.238956,-10.893710,0.033043,0.955943,1.110513,1.005039,8.546047,0.610735,10.095680,1.019345,16.71051,0.952560,609.8846,0.475136,1317.956,0.256875,-141.02610,-0.608951,1612.576,0.212125,900.7050,0.395662,-111.65260,-0.791487,2706.250,0.155800,773.9065,0.492648,-113.50150,-0.762943,-9.636793,-1.066963

### Find distance correlation between intensity features and other features. 



In [33]:
import dcor
correlated_features_d = {}


features = ['duration']

other_features = list(set(df.columns)-set(features)-set(['sid', 'token', 'target', 'filename', 'task']))


for x1 in features:
    for x2 in other_features:
        # if x2 in correlated_features.keys():
        #     continue #skip
        # else:
            dcor_result = float(dcor.distance_correlation(df[x1], df[x2]))
            if x2 not in correlated_features_d.keys():
                correlated_features_d[x2] = dcor_result
            elif x2 in correlated_features_d.keys():
                if dcor_result>correlated_features_d.get(x2):
                    # save the highest correlation between intensity features and nonintensity ones
                    correlated_features_d[x2] = dcor_result
                else:
                    # save the prior correlation
                    continue
            
            
    


In [34]:
correlated_features = pd.DataFrame(correlated_features_d, index = ['dcor']).T.reset_index().sort_values('dcor')[::-1]
correlated_features.columns = ['other_features', 'dcor']
correlated_features


,other_features,dcor
57,equivalentSoundLevel_dBp,0.755525
60,spectralFlux_sma3_amean,0.724038
17,loudness_sma3_stddevRisingSlope,0.707983
18,loudness_sma3_amean,0.702836
25,spectralFluxV_sma3nz_amean,0.701224
...,...,...
22,mfcc3V_sma3nz_stddevNorm,0.079990
39,hammarbergIndexV_sma3nz_stddevNorm,0.075945
49,mfcc4_sma3_stddevNorm,0.073853
65,mfcc4V_sma3nz_amean,0.073674


In [35]:
print(correlated_features[correlated_features['dcor']>0.3].shape)
print(correlated_features[correlated_features['dcor']>0.4].shape)


(35, 2)
(26, 2)


In [36]:
print('remaining vars', 88-35)
print('remaining vars', 88-26)

remaining vars 53
remaining vars 62


In [37]:
correlated_with_duration = correlated_features[correlated_features['dcor']>0.3].round(2)
correlated_with_duration = correlated_with_duration.reset_index(drop = True)

correlated_with_duration.to_csv(output_dir+'uncorrelated_dcor-030_duration.csv')
correlated_with_duration

,other_features,dcor
0,equivalentSoundLevel_dBp,0.76
1,spectralFlux_sma3_amean,0.72
2,loudness_sma3_stddevRisingSlope,0.71
3,loudness_sma3_amean,0.70
4,spectralFluxV_sma3nz_amean,0.70
5,spectralFluxUV_sma3nz_amean,0.70
6,loudness_sma3_percentile80.0,0.69
7,slopeUV500-1500_sma3nz_amean,0.67
8,loudness_sma3_percentile50.0,0.67
9,loudness_sma3_meanRisingSlope,0.65


In [38]:



correlated_with_intensity = pd.read_csv(output_dir+'uncorrelated_dcor-030_intensity.csv', index_col = 0)

correlated_with_intensity



correlated_with_both = correlated_with_duration.merge(correlated_with_intensity, on='other_features', suffixes=('_duration', '_intensity'), how='outer')

correlated_with_both


,other_features,dcor_duration,dcor_intensity
0,equivalentSoundLevel_dBp,0.76,NaN
1,spectralFlux_sma3_amean,0.72,0.90
2,loudness_sma3_stddevRisingSlope,0.71,NaN
3,loudness_sma3_amean,0.70,NaN
4,spectralFluxV_sma3nz_amean,0.70,0.88
5,spectralFluxUV_sma3nz_amean,0.70,0.80
6,loudness_sma3_percentile80.0,0.69,NaN
7,slopeUV500-1500_sma3nz_amean,0.67,0.58
8,loudness_sma3_percentile50.0,0.67,NaN
9,loudness_sma3_meanRisingSlope,0.65,NaN


In [39]:
# insert 1 in dcor with intensity features
correlated_with_both = correlated_with_both.reset_index(drop = True)
correlated_with_both.index = correlated_with_both['other_features'].values
correlated_with_both =  correlated_with_both.drop('other_features', axis=1)
intensity_features = ['loudness_sma3_amean',
       'loudness_sma3_stddevNorm', 'loudness_sma3_percentile20.0',
       'loudness_sma3_percentile50.0', 'loudness_sma3_percentile80.0',
       'loudness_sma3_pctlrange0-2', 'loudness_sma3_meanRisingSlope',
       'loudness_sma3_stddevRisingSlope', 'loudness_sma3_meanFallingSlope',
       'loudness_sma3_stddevFallingSlope','loudnessPeaksPerSec','equivalentSoundLevel_dBp',
                      'HNRdBACF_sma3nz_amean','HNRdBACF_sma3nz_stddevNorm',]

for feature in correlated_with_both.index:
    if feature in intensity_features:
        correlated_with_both.loc[feature, 'dcor_intensity'] = 1

correlated_with_both

,dcor_duration,dcor_intensity
equivalentSoundLevel_dBp,0.76,1.00
spectralFlux_sma3_amean,0.72,0.90
loudness_sma3_stddevRisingSlope,0.71,1.00
loudness_sma3_amean,0.70,1.00
spectralFluxV_sma3nz_amean,0.70,0.88
spectralFluxUV_sma3nz_amean,0.70,0.80
loudness_sma3_percentile80.0,0.69,1.00
slopeUV500-1500_sma3nz_amean,0.67,0.58
loudness_sma3_percentile50.0,0.67,1.00
loudness_sma3_meanRisingSlope,0.65,1.00


In [40]:
correlated_with_both.to_csv(output_dir+'uncorrelated_dcor-030_both.csv')

In [41]:
all_features = correlated_features['other_features']

In [42]:
threshold = 0.3
filename = 'uncorrelated_dcor-030_both'


correlated_both_030 = correlated_with_both[(correlated_with_both['dcor_duration']>=threshold) | (correlated_with_both['dcor_intensity']>=threshold)].index

# all_features
# uncorrelated_features 
len(correlated_both_030)

58

In [43]:
correlated_with_both.shape

(58, 2)

In [44]:
88-45

43

In [45]:
threshold = 0.4
filename = 'uncorrelated_dcor-040_both'


correlated_both_040 = correlated_with_both[(correlated_with_both['dcor_duration']>=threshold) | (correlated_with_both['dcor_intensity']>=threshold)].index

# all_features
# uncorrelated_features 
len(correlated_both_040)


45

In [46]:
output_dir

'./data/output/bias_mitigation/'

In [24]:
egemaps_features = correlated_features['other_features'].values
uncorrelated_above_03 = set(egemaps_features) - set(correlated_both_030)
len(uncorrelated_above_03)

30

In [48]:
%%time


models = [
    LogisticRegressionCV(solver='liblinear', penalty = 'l1', max_iter = 100),
    MLPClassifier(alpha = 1, max_iter= 1000),
    RandomForestClassifier(n_estimators= 100),
    SGDClassifier(loss='log', penalty="elasticnet", early_stopping=True, max_iter = 5000),
]


names = ['LogisticRegressionCV', "MLPClassifier","RandomForestClassifier",'SGDClassifier']


from sklearn.model_selection import train_test_split




toy = False


for threshold in [0.3, 0.4]:

  for null_model in [True, False]:
      print('\npermute', null_model)
      # Create DFs for each independent variable

      if threshold == 0.3:
        variables = correlated_both_030
        filename = 'uncorrelated_below_dcor-030_both'
      elif threshold == 0.4:
        variables = correlated_both_040
        filename = 'uncorrelated_below_dcor-040_both'

          

      X = df[variables].values
      y = df['target'].values


      if toy:
        n_bootstraps = 3
      else:
        n_bootstraps = 50

      if null_model:
          y = np.random.permutation(y) #CHECK


      y_pred_all = {}
      roc_auc_all = {}
      for model, name in zip(models, names):
        y_pred_all[name] = []
        roc_auc_all[name] = []
        pipe = Pipeline(steps=[
                ('scaler', StandardScaler()), 
                ('model', model)])

        ## Performing bootstrapping
        for i in range(n_bootstraps):
            #Split the data into training and testing set

            # Chaning the seed value for each iteration
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42+i)

        # for train_index, test_index in bs:
        #   X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
            pipe.fit(X_train,y_train)

            # # Evaluate
            # y_proba = pipe.predict_proba(X_test)       # Get predicted probabilities
            # y_proba_1 = y_proba[:,1]
            # y_pred = np.argmax(y_proba, axis=1) 
            # roc_auc = metrics.roc_auc_score(y_test, y_proba_1)  
                      
            y_pred = pipe.predict(X_test) 
            roc_auc = metrics.roc_auc_score(y_test, y_pred)  # ROC AUC takes probabilities but here we match what pydra-ml does: https://github.com/nipype/pydra-ml/issues/56

            y_pred_all[name].append(y_pred)
            roc_auc_all[name].append(roc_auc)

      results_i = []
      for name in ['LogisticRegressionCV','MLPClassifier','RandomForestClassifier','SGDClassifier']:
        scores = roc_auc_all.get(name)
        roc_auc_median = np.round(np.median(scores),2)
        roc_auc_5 = np.round(np.percentile(scores, 5),2)
        roc_auc_95 = np.round(np.percentile(scores, 95),2)
        results_str = f'{roc_auc_median} ({roc_auc_5}–{roc_auc_95}; )'
        results_str = results_str.replace('0.', '.')
        results_i.append([name, results_str])

        if null_model:
          print(name, str(roc_auc_median).replace('0.', '.'))
      if not null_model:
          results_i_df = pd.DataFrame(results_i, ).T
          display(results_i_df)
          results_i_df.to_csv(output_dir+f'results_{filename}_permute-{null_model}_duration_{ts}.csv')




      # pd.DataFrame(y_pred_all)



permute True


/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/

LogisticRegressionCV .5
MLPClassifier .51
RandomForestClassifier .52
SGDClassifier .5

permute False


/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/

,0,1,2,3
0,LogisticRegressionCV,MLPClassifier,RandomForestClassifier,SGDClassifier
1,.88 (.83–.91; ),.89 (.86–.93; ),.89 (.84–.92; ),.86 (.78–.9; )



permute True


/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/

LogisticRegressionCV .5
MLPClassifier .52
RandomForestClassifier .54
SGDClassifier .51

permute False


,0,1,2,3
0,LogisticRegressionCV,MLPClassifier,RandomForestClassifier,SGDClassifier
1,.85 (.79–.9; ),.9 (.86–.93; ),.88 (.82–.93; ),.8 (.7–.88; )


CPU times: user 25min 21s, sys: 7min 52s, total: 33min 14s
Wall time: 23min
